In [ ]:
pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk]

In [ ]:
pip install -U -q "google-genai"

In [ ]:
pip install --quiet gradio

In [ ]:
# --- Set environment variables
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = "content-creation-agent-468223"    #Name of Your Project
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"                     #Location
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"                        #Keep it to true to use Vertex AI
os.environ["GOOGLE_CLOUD_BUCKET"] = "gs://launchpad_marketing"          #Bucket used for staging
os.environ["GEMINI_VERSION"] = "gemini-2.0-flash"                       #Gemini version to use for Agents

In [ ]:
# --- Create Blog Post LLM Agent
from google.adk.agents import LlmAgent
from google.genai import types
from vertexai.preview.reasoning_engines import AdkApp

# --- Set safety settings:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.7,
   max_output_tokens= 800,
   top_p= 0.95,
)


blog_post_instructions = """
            You are a junior copywriter tasked with creating a product announcement blog post.
            Goal: Write an engaging, well-structured blog post of 500–600 words based on the provided product brief.
            Your responsibilities:
                1. Accept a simple product brief with:
                    - Product Name
                    - Short Description
                    - Key Features (bullet list)
                    - Target Audience
                2. Craft the content in a clear, accessible tone suited for the target audience.
                3. Structure the blog post with the following sections:
                    - Title: Compelling and relevant to the product launch.
                    - Introduction: Briefly introduce the product and hook the reader’s interest.
                    - Body: Expand on the product’s benefits, use cases, and key features in 2–3 short paragraphs.
                    - Conclusion: Summarize the main selling points and end with a clear call-to-action.
                4. Avoid overly technical language unless the target audience is highly technical.
                5. Maintain a friendly yet professional tone.
            Output Format:
                - Provide the blog post as markdown with section headings.
                - Do not include any placeholder text — fully write out all sections.
                - Keep word count between 500 and 600 words.
            """

blog_post_agent = LlmAgent(
    name="BlogPostAgent", 
    model = os.getenv("GEMINI_VERSION"),
    description=("Agent that writes a 400-500 word announcement blog post"),
    instruction=blog_post_instructions, 
    generate_content_config=generate_content_config,
    output_key="result")

blog_app = AdkApp(agent=blog_post_agent)
print("✅ BlogPostAgent is ready to be deployed")

In [ ]:
# --- Local test of Blog Agent
for event in blog_app.stream_query(
    user_id="user_123",
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)

In [ ]:
# --- Create Social Media LLM Agent
from google.adk.agents import LlmAgent, ParallelAgent
from google.genai import types
from vertexai.preview.reasoning_engines import AdkApp

# --- Set safety settings:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.7,
   max_output_tokens= 800,
   top_p= 0.90,
)


social_media_instructions = """
        You are a social media coordinator responsible for creating short-form promotional posts for a new product launch.

        Goal: 
        - Based on the provided product brief, generate engaging, platform-specific posts. 
        - If the brief does NOT specify platforms, default to LinkedIn, Twitter, and Instagram. 
        - If the user explicitly requests one or more platforms, generate posts only for those platforms.

        Your responsibilities:
        1. Accept a simple product brief with:
           - Product Name
           - Short Description
           - Key Features (bullet list)
           - Target Audience
           - (Optional) Social Platforms: one or more platforms requested by the user (e.g., LinkedIn, Twitter, Instagram, TikTok, Facebook, YouTube Shorts, Reddit, Threads, Pinterest).
        2. Select target platforms:
           - If platforms are provided in the brief or prompt, use exactly those.
           - Otherwise, default to: LinkedIn, Twitter, Instagram.
           - Normalize synonyms: treat "X" or "X/Twitter" as "Twitter".
        3. Create distinct, on-brand short-form posts tailored to each selected platform:
           - LinkedIn: Professional tone, highlights business value, up to 100 words.
           - Twitter: Punchy, attention-grabbing, max 280 characters.
           - Instagram: Engaging, visually descriptive caption with relevant hashtags (max 10).
           - If a different platform is requested, adapt tone/length to that platform’s norms and limits.
        4. Ensure each post reflects the platform’s tone and constraints. Avoid generic filler; make every word impactful.

        Output Format (strict):
        - Return a single JSON object with one key per selected platform.
        - Use exact platform keys (e.g., "LinkedIn", "Twitter", "Instagram", "TikTok", "Facebook", "YouTube Shorts", "Reddit", "Threads", "Pinterest").
        - Do NOT wrap the JSON in code fences or add extra commentary.

        Default example shape (when no platforms are specified):
        {
          "LinkedIn": "Full LinkedIn post text",
          "Twitter": "Full Twitter post text",
          "Instagram": "Full Instagram caption text"
        }

        Additional Notes:
        - Ensure all content is original and aligned to the specified target audience.
        - Include a clear hook or CTA where appropriate.
        - For Instagram, ensure hashtags are relevant and non-repetitive.
        - No placeholders like [link]; if a URL is not provided, omit it gracefully.
        """

social_media_agent = LlmAgent(
    name="SocialMediaAgent", 
    model = os.getenv("GEMINI_VERSION"),
    description=("Agent that generates multi-platform short-form posts in structured JSON."),
    instruction=social_media_instructions, 
    generate_content_config=generate_content_config,
    output_key="result")

social_app = AdkApp(agent=social_media_agent)
print("✅ SocialMediaAgent is ready to be deployed")

In [ ]:
#--- Local Test of Social Media Agent
for event in social_app.stream_query(
    user_id="user_123",
    message="""
    I want to announce a new product called VisionLink Pro.
    It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
    The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
    The target audience is remote teams, global companies, and project managers.
    """,):
    print(event)

In [ ]:
# --- Agent that will call both Blog and Social Agents in parallel (Used by Coordinator)
gather_agent = ParallelAgent(name="InfoGatherer", sub_agents=[blog_post_agent, social_media_agent])

In [ ]:
# --- Create Coordinator LLM Agent -> SubAgent InfoGatherer
campaign_coordinator_system_instructions = """
You are the Campaign Coordinator, an orchestrator agent that manages the end-to-end content creation workflow for a product launch.

Goal
Given a product brief, call InfoGatherer and return a single, cohesive campaign package as either:
- A unified JSON object, or
- A formatted Markdown file (preferred for human review).

Inputs
A simple product brief containing:
- Product Name
- Short Description
- Key Features (bullet list)
- Target Audience

Available Tools / Sub-Agents
- InfoGatherer -> Parallel Agent that returns the results of both BlogPostAgent and SocialMediaAgent

Workflow
1) Validate Brief
   - Ensure required fields are present and non-empty.
   - Normalize keys and trim whitespace. If anything critical is missing, add it under notes.missing_fields in the final output.

2) Plan & Call Sub-Agents
   - Call InfoGatherer unless explicitly disabled in the brief.
   - Pass the brief verbatim and include goal hints:
     * BlogPostAgent: "Produce a 500–600 word launch post."
     * SocialMediaAgent: "Produce Social Media posts as specified."
   - On failure, retry once. If still failing, continue and record under notes.errors.

3) Consolidate & Harmonize
   - Check social posts align factually with the blog post and brief (no contradictions).
   - Unify tone: friendly, professional, concise (unless the audience is explicitly technical).
   - Extract a single launch CTA and ensure it appears across channels (rephrased as needed).

4) Output
   - If output_format == "json", return the following JSON shape:
     {
       "brief": {
         "product_name": "...",
         "short_description": "...",
         "key_features": ["..."],
         "target_audience": "..."
       },
       "assets": {
         "blog_post_markdown": "## Title...\\n...",
         "social_posts": {
           "LinkedIn": "...",
           "Twitter": "...",
           "Instagram": "..."
         }
       },
       "consistency_check": {
         "facts_confirmed": ["..."],
         "potential_gaps": ["..."]
       },
       "notes": {
         "missing_fields": [],
         "errors": []
       }
     }

   - If output_format == "markdown" (default), return:
     "# Campaign Package: <Product Name>\\n\\n## Blog Post\\n<full blog post markdown>\\n\\n## Social Posts\\n- **LinkedIn:** <text>\\n- **Twitter:** <text>\\n- **Instagram:** <caption>\\n\\n---\\n### Consistency Check\\n- Facts confirmed: …\\n- Potential gaps: …\\n\\n### Notes\\n- Missing fields: …\\n- Errors: …"

Quality Bar
- No hallucinated facts; only use the brief and sub-agent outputs.
- Respect platform constraints (e.g., Twitter ≤ 280 chars).
- Remove repetitive hashtags and unnecessary jargon.
- Ensure spelling and grammar are clean across all assets.

Failure Handling
- If a sub-agent returns empty or invalid output:
  - Retry once.
  - If still invalid, omit that asset, record under notes.errors, and proceed.
"""

coordinator_agent = LlmAgent(
    name="CoordinatorAgent",
    model = os.getenv("GEMINI_VERSION"),
    instruction=campaign_coordinator_system_instructions,
    description="Campaign Coordinator",
    sub_agents=[gather_agent] #parallel agent that executes both Blog and Social Agents
)

coordinator_app = AdkApp(agent=coordinator_agent)
print("✅ CoordinatorAgent is ready to be deployed")

In [ ]:
#--- Local test for Coordinator Agent
for event in coordinator_app.stream_query(
    user_id="user_123",
    message=""" Product Name: VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                Platforms: LinkedIn, Instagram
                """,):
    print(event)

In [ ]:
# --- Deploy all Agents in Vertex AI Agent Engine (Wait for Creation of all agents before moving on...)
import vertexai

from vertexai import agent_engines

# --- Init Vertex AI Environment ---
vertexai.init(
    project= os.getenv("GOOGLE_CLOUD_PROJECT"),
    location= os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket= os.getenv("GOOGLE_CLOUD_BUCKET"), 
)

# --- Deploy Blog Post Agent ---
remote_blog_agent = agent_engines.create(
    blog_app,
    display_name="Blog Post Agent",
    description="Writes 400–600 word product announcement posts in Markdown.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],  #"cloudpickle==3.1.1","pydantic==2.11.7"
)
print("✅ Blog Post Agent created on Vertex AI Agent Engine")

# --- Deploy Social Media Agent ---
remote_social_agent = agent_engines.create(
    social_app,
    display_name="Social Media Agent",
    description="Generates multi-platform short-form posts in structured JSON from a product brief.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"], 
)
print("✅ Social Media Agent created on Vertex AI Agent Engine")

# --- Deploy Campaign Coordinator Agent ---
remote_campaign_coordinator_agent = agent_engines.create(
    coordinator_app,
    display_name="Campaign Coordinator Agent",
    description="Orchestrates Blog Post and Social Media Agents to produce a unified campaign package.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],
)
print("✅ Campaign Coordinator Agent created on Vertex AI Agent Engine")

In [ ]:
# --- Test consolidation of Blog + Social Media Agent
remote_session = remote_campaign_coordinator_agent.create_session(user_id="user_123")

for event in remote_campaign_coordinator_agent.stream_query(
    user_id="user_123",
    session_id = remote_session["id"],
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration. 
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video. 
                The target audience is remote teams, global companies, and project managers."""
   ):
    print(event)

In [ ]:
# gradio_agent_console.py
# ------------------------------------------------------------
# Gradio UI for 3 Vertex AI Agent Engine agents (ADK style)
# - Left pane: Agent selector, sample prompts, prompt box, Send
# - Right pane: Chatbot (per-agent history), session-aware
# - Controls: New Session (per-agent), Clear Chat (keep session)

# Docs:
#   - Use an agent (query/stream_query): https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview
#   - ADK sessions (create_session/stream_query with session_id): https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk
# ------------------------------------------------------------

import os
import json
import gradio as gr
import vertexai
from vertexai import agent_engines

# ---------------------- CONFIG ------------------------------
# Set these to your deployed Agent Engine resource names.
BLOG_AGENT_RESOURCE        = remote_blog_agent.resource_name
SOCIAL_AGENT_RESOURCE      = remote_social_agent.resource_name
COORDINATOR_AGENT_RESOURCE = remote_campaign_coordinator_agent.resource_name

AGENT_LABELS = {
    "📝 Blog Post Agent": BLOG_AGENT_RESOURCE,
    "📣 Social Media Agent": SOCIAL_AGENT_RESOURCE,
    "🧭 Campaign Coordinator Agent": COORDINATOR_AGENT_RESOURCE,
}

# A stable user_id helps the Agent Engine associate sessions with your user.
DEFAULT_USER_ID = "UI-user"

# ---- Simple labels for the dropdown; full text inserted into the Prompt box ----
SAMPLE_PROMPT_MAP = {
    "NimbusNote Pro (consultants)": (
        "Create launch content using this brief: The product is NimbusNote Pro, an AI-powered note-taking app that helps "
        "consultants organize client work, auto-summarize meetings, and draft deliverables faster. Key features include AI summaries "
        "with action items, cross-device sync with offline mode, project spaces with permissions, export to PDF/DOCX/Markdown, and "
        "ready-to-use templates for proposals and SOWs. The target audience is management consultants and knowledge workers in SMB "
        "and mid-market firms. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
    "AeroLens 2.0 (field service)": (
        "Create launch content using this brief: The product is AeroLens 2.0, lightweight AR glasses for field technicians that "
        "overlay step-by-step guidance and capture hands-free photos and video. Key features include on-device offline manuals, "
        "voice commands with gesture control, remote expert assistance, a rugged design with all-day battery, and work order sync "
        "with popular EAM/CMMS tools. The target audience is utilities, manufacturing, and field service teams. For social, focus on "
        "LinkedIn, X/Twitter, and Instagram."
    ),
    "SparkBrew (home office coffee)": (
        "Create launch content using this brief: The product is SparkBrew, a smart coffee maker for home offices with precise "
        "temperature control and scheduled brewing via a mobile app. Key features include barista-grade temperature and bloom control, "
        "brew scheduling with reminders, custom profiles by roast, cleaning alerts with a maintenance log, and a compact low-noise "
        "design. The target audience is remote professionals and small office teams. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
    "FleetSense (fleet IoT)": (
        "Create launch content using this brief: The product is FleetSense, an IoT fleet platform that monitors vehicle health, "
        "driver behavior, and routing to reduce fuel costs and downtime. Key features include real-time GPS with geofencing, predictive "
        "maintenance alerts, driver safety scoring, fuel optimization insights, and open APIs for TMS/ERP integration. The target "
        "audience is logistics managers and operations leaders in fleet-heavy businesses. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
    "ClaraPay (AP automation)": (
        "Create launch content using this brief: The product is ClaraPay, a B2B payments automation solution that streamlines invoice "
        "capture, approvals, and reconciliation within existing accounting stacks. Key features include OCR with AI data extraction, "
        "multistep approvals with thresholds, vendor management with 1099 support, automated three-way matching, and ERP connectors for "
        "QuickBooks, NetSuite, and Xero. The target audience is SMB and mid-market finance teams in AP/AR. For social, focus on "
        "LinkedIn, X/Twitter, and Instagram."
    ),
    "Gardenie One (indoor smart garden)": (
        "Create launch content using this brief: The product is Gardenie One, an indoor smart garden that grows herbs and greens "
        "year-round with automatic lighting and watering. Key features include adaptive LED grow cycles, a self-watering reservoir, "
        "app-based growth tips and reminders, non-GMO seed pods, and dishwasher-safe trays. The target audience is health-conscious home "
        "cooks and apartment dwellers. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
    "CodeZen AI (Python devs)": (
        "Create launch content using this brief: The product is CodeZen AI, an AI pair programmer for Python that suggests tests, "
        "explains errors, and enforces a team’s style guide. Key features include inline suggestions and refactors, unit-test generation "
        "with coverage hints, CI-ready linting presets (Black and Flake8), repo-aware context windows, and security checks for secrets and SQL. "
        "The target audience is Python engineering teams at startups and SaaS companies. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
    "LumaDesk Glow (smart desk lamp)": (
        "Create launch content using this brief: The product is LumaDesk Glow, a smart LED desk lamp that adapts brightness and color "
        "temperature to your circadian rhythm for better focus. Key features include automatic ambient light sensing, a warm-to-cool "
        "temperature range, eye-strain reduction modes, app and voice assistant control, and energy usage insights. The target audience is "
        "students, designers, and remote workers who need healthier work lighting. For social, focus on LinkedIn, X/Twitter, and Instagram."
    ),
}

# If all agents share the same samples, just reuse the same label list:
SAMPLE_LABELS = list(SAMPLE_PROMPT_MAP.keys())

# ------------------ Agent Helpers ---------------------------
def _get_agent(resource_name):
    """Return a remote agent object from Agent Engine."""
    return agent_engines.get(resource_name)

def _create_session(agent, user_id: str) -> str:
    """
    Create a managed session via ADK-style op.
    Returns session_id string.
    """
    # ADK 'Use' doc exposes create_session(user_id=...) on the agent handle.
    session = agent.create_session(user_id=user_id)
    # session may be a dict or object with 'id'
    if isinstance(session, dict):
        return session.get("id") or session.get("session_id") or ""
    return getattr(session, "id", "") or getattr(session, "session_id", "")

def _extract_stream_output(event) -> str:
    """
    Extract the most useful text from a streamed event.
    For ADK stream_query, the final event often includes 'output'.
    Earlier events may include 'actions' or 'steps'.
    """
    # event might be dict-like already
    if hasattr(event, "to_dict"):
        try:
            event = event.to_dict()
        except Exception:
            event = dict(event)
    elif not isinstance(event, dict):
        # Try best effort
        try:
            event = json.loads(str(event))
        except Exception:
            return str(event)

    # Priority: 'output' field
    out = event.get("output")
    if isinstance(out, (str, int, float)):
        return str(out)
    if isinstance(out, (dict, list)):
        try:
            return json.dumps(out, indent=2, ensure_ascii=False)
        except Exception:
            return str(out)

    # Fallbacks seen in some templates
    for key in ("result", "output_text", "message", "text"):
        val = event.get(key)
        if isinstance(val, str) and val.strip():
            return val

    # If there's a 'content' shape with 'parts'
    content = event.get("content")
    if isinstance(content, dict):
        parts = content.get("parts") or []
        texts = []
        for p in parts:
            t = p.get("text")
            if isinstance(t, str) and t.strip():
                texts.append(t.strip())
        if texts:
            return "\n".join(texts)

    # Nothing obvious
    return ""

def _stream_agent_reply(agent, user_id: str, session_id: str, message: str) -> str:
    """
    Call stream_query and return the final consolidated text.
    We join partial readable chunks and prefer the final 'output' if present.
    """
    final_text = ""
    collected = []
    # ADK 'Use' doc supports stream_query(user_id=..., session_id=..., message=...)
    for ev in agent.stream_query(user_id=user_id, session_id=session_id, message=message):
        chunk = _extract_stream_output(ev)
        if chunk:
            collected.append(chunk)
            final_text = chunk  # last non-empty chunk usually the final answer

    # If we saw only partials and no clear final, return the concatenation.
    if final_text:
        return final_text
    if collected:
        return "\n".join(collected)
    return "[no output]"

# ------------------ Gradio Callbacks ------------------------
def on_agent_change(agent_label: str):
    """Update samples and (optionally) placeholder when agent changes."""
    return gr.update(choices=SAMPLE_LABELS, value=None), gr.update(value="")

def on_sample_label_change(sample_label: str):
    # insert the full prompt text into the textbox
    return SAMPLE_PROMPT_MAP.get(sample_label, "")

def ensure_session_for_agent(agent_label: str, sessions_state: dict, user_id: str):
    """Fetch or create a session for the selected agent."""
    sessions_state = sessions_state or {}
    sid = sessions_state.get(agent_label)
    if sid:
        return sid, sessions_state  # re-use
    agent = _get_agent(AGENT_LABELS[agent_label])
    new_sid = _create_session(agent, user_id=user_id)
    sessions_state[agent_label] = new_sid
    return new_sid, sessions_state

# ------------------Formatting----------------------
import re, json

CODE_FENCE_JSON_RE = re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE)

def _maybe_parse_json(text: str):
    """Return parsed JSON (dict/list) if text contains valid JSON; else None.
    Handles ```json fences``` and bare JSON blobs."""
    if not isinstance(text, str) or not text.strip():
        return None

    # 1) Try fenced ```json ... ```
    m = CODE_FENCE_JSON_RE.search(text)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass

    # 2) Try full text as JSON
    try:
        return json.loads(text)
    except Exception:
        pass

    # 3) Fallback: find the biggest {...} and try
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        try:
            return json.loads(candidate)
        except Exception:
            pass

    return None

def _format_social_posts_md(data: dict) -> str:
    li = data.get("LinkedIn") or data.get("linkedin")
    tw = data.get("Twitter") or data.get("X") or data.get("x_twitter") or data.get("twitter")
    ig = data.get("Instagram") or data.get("instagram")
    out = ["### Social Posts"]
    if li: out.append(f"- **LinkedIn:** {li}")
    if tw: out.append(f"- **Twitter/X:** {tw}")
    if ig: out.append(f"- **Instagram:** {ig}")
    return "\n".join(out) if len(out) > 1 else "```json\n" + json.dumps(data, indent=2, ensure_ascii=False) + "\n```"

def _format_coordinator_md(pkg: dict) -> str:
    brief = pkg.get("brief") or {}
    assets = pkg.get("assets") or {}
    blog = assets.get("blog_post_markdown") or assets.get("blog_post") or ""
    social = assets.get("social_posts") or {}

    title = brief.get("product_name") or brief.get("Product Name") or "Campaign Package"
    parts = [f"# Campaign Package: {title}"]

    if blog:
        parts += ["", "## Blog Post", blog.strip()]

    if social:
        parts += ["", _format_social_posts_md(social)]

    # Optional diagnostics
    cc = pkg.get("consistency_check")
    if cc:
        parts += ["", "### Consistency Check", "```json", json.dumps(cc, indent=2, ensure_ascii=False), "```"]

    notes = pkg.get("notes")
    if notes:
        parts += ["", "### Notes", "```json", json.dumps(notes, indent=2, ensure_ascii=False), "```"]

    return "\n".join(parts)

def _format_for_display(agent_label: str, raw_text: str) -> str:
    """Turn any agent output into nice markdown for the Chatbot."""
    data = _maybe_parse_json(raw_text)

    # If JSON detected, format intelligently
    if data is not None:
        # Social posts shape
        if isinstance(data, dict) and any(k in data for k in ("LinkedIn", "linkedin", "Twitter", "X", "Instagram")):
            return _format_social_posts_md(data)

        # Coordinator package shape
        if isinstance(data, dict) and ("assets" in data or "brief" in data):
            return _format_coordinator_md(data)

        # Unknown JSON → pretty-print
        return "```json\n" + json.dumps(data, indent=2, ensure_ascii=False) + "\n```"

    # No JSON → likely plain markdown/text (e.g., Blog agent)
    return raw_text or "[no output]"


def send_message(agent_label: str, prompt_text: str, chat_map: dict, sessions_state: dict, user_id: str):
    """
    Send the message to the selected agent using the persisted session.
    Returns updated chatbot content and states.
    """
    chat_map = chat_map or {}
    chat = _to_messages(chat_map.get(agent_label, []))

    if not prompt_text or not prompt_text.strip():
        return chat, chat_map, sessions_state, gr.update(value="")  # no change

    # Ensure session
    session_id, sessions_state = ensure_session_for_agent(agent_label, sessions_state, user_id)
    agent = _get_agent(AGENT_LABELS[agent_label])

    # Call the agent (streaming) and collect final text
    try:
        raw_reply = _stream_agent_reply(agent, user_id=user_id, session_id=session_id, message=prompt_text.strip())
        reply_text = _format_for_display(agent_label, raw_reply)
    except Exception as e:
        reply_text = f"[stream_query failed] {e}"

    chat.append({"role": "user", "content": prompt_text})
    chat.append({"role": "assistant", "content": reply_text})
    chat_map[agent_label] = chat
    return chat, chat_map, sessions_state, gr.update(value="")

def new_session(agent_label: str, chat_map: dict, sessions_state: dict, user_id: str):
    """Create a fresh session for the selected agent and clear only that agent's chat."""
    chat_map = chat_map or {}
    sessions_state = sessions_state or {}

    agent = _get_agent(AGENT_LABELS[agent_label])
    sid = _create_session(agent, user_id=user_id)

    sessions_state[agent_label] = sid
    chat_map = chat_map or {}
    chat_map[agent_label] = []
    return chat_map[agent_label], chat_map, sessions_state

def clear_chat_only(agent_label: str, chat_map: dict):
    """Clear chat history for the selected agent but keep the session."""
    chat_map = chat_map or {}
    chat_map[agent_label] = []
    return chat_map[agent_label], chat_map

def _to_messages(history):
    if not history: return []
    if isinstance(history, list) and (not history or isinstance(history[0], dict)):
        return history
    msgs = []
    for u, a in history:
        if isinstance(u, str) and u.strip():
            msgs.append({"role": "user", "content": u})
        if isinstance(a, str) and a.strip():
            msgs.append({"role": "assistant", "content": a})
    return msgs


# ---------------------- UI -------------------------------
with gr.Blocks(title="Eddy's LaunchPad: Agent Engine Console", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🚀 LaunchPad — Agent Engine Console\nInteract with your deployed agents.")

    # Global state:
    # - chat_map: { agent_label -> list[(user, bot), ...] }
    # - sessions_state: { agent_label -> session_id }
    chat_map = gr.State({})
    sessions_state = gr.State({})
    user_id_box = gr.Textbox(value=DEFAULT_USER_ID, label="User ID (for sessions)", interactive=True)

    with gr.Row():
        with gr.Column(scale=1, min_width=340):
            agent_dd = gr.Dropdown(
                choices=list(AGENT_LABELS.keys()),
                value="🧭 Campaign Coordinator Agent",
                label="Select Agent",
                interactive=True,
            )
            sample_dd = gr.Dropdown(
                choices=SAMPLE_LABELS,
                label="Prompt Samples",
                interactive=True,
                value=None  # start blank
            )
            prompt_tb = gr.Textbox(
                label="Prompt",
                placeholder="Type your request for the selected agent…",
                lines=8,
            )
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                new_sess_btn = gr.Button("New Session (Selected Agent)", variant="secondary")
            clear_btn = gr.Button("Clear Chat (Selected Agent)", variant="secondary")

        with gr.Column(scale=2, min_width=540):
            session_id_display = gr.Textbox(label="Current Session ID (Selected Agent)", interactive=False)
            chatbot = gr.Chatbot(
                label="Conversation",
                type="messages",
                height=520
            )

    # --- Wiring ---
    agent_dd.change(fn=on_agent_change, inputs=[agent_dd], outputs=[sample_dd, prompt_tb])
    sample_dd.change(fn=on_sample_label_change, inputs=[sample_dd], outputs=[prompt_tb])

    # Ensure we show the current session id on focus/agent change:
    def refresh_session_id(agent_label, sessions_state, user_id):
        sid, sessions_state = ensure_session_for_agent(agent_label, sessions_state, user_id)
        return sid, sessions_state
    agent_dd.change(fn=refresh_session_id, inputs=[agent_dd, sessions_state, user_id_box], outputs=[session_id_display, sessions_state])

    send_btn.click(
        fn=send_message,
        inputs=[agent_dd, prompt_tb, chat_map, sessions_state, user_id_box],
        outputs=[chatbot, chat_map, sessions_state, prompt_tb],
    ).then(
        fn=refresh_session_id,
        inputs=[agent_dd, sessions_state, user_id_box],
        outputs=[session_id_display, sessions_state]
    )

    new_sess_btn.click(
        fn=new_session,
        inputs=[agent_dd, chat_map, sessions_state, user_id_box],
        outputs=[chatbot, chat_map, sessions_state],
    ).then(
        fn=refresh_session_id,
        inputs=[agent_dd, sessions_state, user_id_box],
        outputs=[session_id_display, sessions_state]
    )

    clear_btn.click(
        fn=clear_chat_only,
        inputs=[agent_dd, chat_map],
        outputs=[chatbot, chat_map],
    )

# To run:
demo.launch(share=True)

In [ ]:
#--- Cleanup Turn Off UI
demo.close()

In [ ]:
#--- Cleanup Delete Deployed Agents
remote_blog_agent.delete(force=True)
remote_social_agent.delete(force=True)
remote_campaign_coordinator_agent.delete(force=True)